## Import packages

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import math
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import numpy as np
import json
import shutil

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = '/content/drive/My Drive/AI PROJECTS/GNN/dataset'

## Define Hyperparameters

In [ ]:
# N_list = [10000, 20]
# window_list = [1, 5, 10] # second

N_list = [30]
window_list = [10] # second

In [ ]:
apps_1 = ['diijam', 'baomoi', 'fptplay', 'iQIYI', 'bigo', 'myradio', 'spotify', 'nhaccuatui', 'soundcloud', 'sachnoiapp', 'phim247', 'popskid']
apps_2 = ['truyenaudiosachnoiviet', 'vieon', 'voizfm', 'tunefm', 'wetv', 'zingmp3', 'truyenaudio', 'baohay24h', 'freefire', 'among_us', 'azar', 'comico', 'nimotv']
apps_3 = ['mangatoon', 'medoctruyen', 'nhacvang', 'noveltoon', 'radiofm', 'tivi247', 'vtvgo', 'tivi24h', 'tinder', 'tinmoi24h', 'tivi360']
apps_4 = ['tiktok', 'linkedin', 'tiki', 'tinhte', 'lotus', 'tivi_truyentranh_webtoon', 'tuoitre_online', 'vietnamworks', 'wallstreet_journal', 'cnn_news', 'bbc_news']
apps_5 = ['twitter', 'weeboo', 'twitch', 'vnexpress', 'topcv', 'toc_chien', 'wesing', 'hago', 'google_meet', 'dubsmash', 'facebook']
apps_6 = ['hahalolo', 'zalo', 'hello_yo', 'dan_tri', 'zoom', 'wikipedia', 'instagram', 'jobway', 'kaka', 'pinterest', 'quora', 'lazada', 'chess', 'cake', 'mobile_legend', 'co_tuong_online']
apps_7 = ['ted', 'telegram', 'starmarker', 'skype', 'soha', 'tango', 'thanhnien', 'snapchat', 'tien_len', 'animal_restaurant', 'bida', 'cho_tot', 'messenger', 'netflix', 'nonolive', 'ola_party', 'may']
apps_8 = ['podcast_player', 'pubg', 'partying', 'kenh14', 'lienquan_mobile', 'likee_lite', 'reddit', 'sendo', 'shopee', 'the_guardian']

remains = ['nhaccuatui', 'soundcloud', 'sachnoiapp', 'phim247', 'popskid']

## Delete empty app folder

In [ ]:
graph_folder = '/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train_graphs'
for N in N_list:
    for window in window_list:
      graph_Nw = os.path.join(graph_folder, 'N{}/t{}'.format(N, window))
      for app in os.listdir(graph_Nw):
        app_folder = os.path.join(graph_Nw, app)
        if len(os.listdir(app_folder)) == 0:
          print(app)
          shutil.rmtree(app_folder)
      print('.......................')

.......................


## Check and remove error app

In [ ]:
"""
Copy samples into training folder and testing folder
"""
train_folder = os.path.join(root_path, '5_3/train_graphs')
test_folder = os.path.join(root_path, '5_3/test_graphs')

with open('/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train_test_info.json', 'r') as f:
  train_test_info = json.load(f)

In [ ]:
def check_graphs(graphs_folder):
  error_apps = []
  unavailable_apps = []

  for app in train_test_info.keys():
    print(app, end='')
    for N in N_list:
      for window in window_list:
        if N==10 and window==10:
          graphs_Nw_folder = os.path.join(graphs_folder, 'N{}/t{}'.format(N, window))

          # create folder for app
          app_folder = os.path.join(graphs_Nw_folder, app)

          filenames = train_test_info[app][0]
          graphs_num = len(filenames)
          if os.path.exists(app_folder):
            # count number of graph
            filename = os.listdir(app_folder)[0]
            path = os.path.join(app_folder, filename)
            df = pd.read_csv(path)
            graphs_count = int(df['graph_id'].iloc[-1])

            print('\n{} - {} = {}'.format(graphs_num, graphs_count, graphs_num - graphs_count), end = '')
            
            if graphs_count < 400:
              error_apps.append(app)
              print(' xxxxxxxx error xxxxxxxx', end='')
              # shutil.rmtree(app_folder)
          else:
              print('\nNOT EXISTS', end='')
              unavailable_apps.append(app)
    
    print('\n.....................................................................')

  unavailable_apps = list(set(unavailable_apps))
  error_apps = list(set(error_apps))

  print('Number of unavailable apps: {}'.format(len(unavailable_apps)))
  print(unavailable_apps)
  print('Number of error apps: {}'.format(len(error_apps)))
  print(error_apps)

In [ ]:
check_graphs(train_folder)
# check_graphs(test_folder)

diijam
643 - 374 = 269 xxxxxxxx error xxxxxxxx
.....................................................................
baomoi
619 - 617 = 2
.....................................................................
fptplay
991 - 505 = 486
.....................................................................
iQIYI
749 - 735 = 14
.....................................................................
bigo
1023 - 988 = 35
.....................................................................
myradio
625 - 487 = 138
.....................................................................
spotify
881 - 866 = 15
.....................................................................
nhaccuatui
1200 - 1200 = 0
.....................................................................
soundcloud
860 - 821 = 39
.....................................................................
sachnoiapp
736 - 611 = 125
.....................................................................
phim247
781 - 498 = 283
...............

## Graph generator functions

Basic reprocessing

In [ ]:
def basic_reprocessing(df, N):
  # remove unknown protocol
  df = df[df['protocol'] != 'unknown']

  # remove dns protocol
  df = df[(df['source_port'] != 53) & (df['destination_port'] != 53) & 
        (df['source_port'] != 5353) & (df['destination_port'] != 5353) &
        (df['source_port'] != 137) & (df['destination_port'] != 137) &
        (df['source_port'] != 67) & (df['destination_port'] != 67) &
        (df['source_port'] != 68) & (df['destination_port'] != 68) &
        (df['source_port'] != 5355) & (df['destination_port'] != 5355)]
  
  # remove IP 239.255.255.250, protocol SSDP
  df = df[(df['source_address'] != '239.255.255.250') & (df['destination_address'] != '239.255.255.250')]

  # convert data: bytes -> KB
  # df['length'] = df['length']/(1000)
  df['length'] = df['length']

  # get server destination column 
  df['des_greater_src'] = df['destination_port'] - df['source_port']
  df1 = df[df['des_greater_src'] > 0]
  df2 = df[df['des_greater_src'] < 0]
  df1['destination'] = df1['source_address']
  df1['port'] = df1['source_port']
  df1['outgoing'] = 0
  df2['destination'] = df2['destination_address']
  df2['port'] = df2['destination_port']
  df2['outgoing'] = 1
  df = pd.concat([df1, df2], ignore_index=True).sort_values(by='time').reset_index(drop=True)

  # merge IP address into port (same tuple (IP, port) - same network destination)
  df['IP_port'] = list(zip(df['destination'], df['port']))

  df = df.drop(['source_address', 'destination_address', 'certificate', 'des_greater_src', 'source_port', 'destination_port', 'destination', 'port'], axis=1)

  # get N network destinations that have the most packets
  df_ = df.groupby(['IP_port'], as_index = False).agg({'length':['count']}).sort_values(by=[('length', 'count')], ascending=False)
  destinations = df_[:N]['IP_port']

  return df[df['IP_port'].isin(destinations)].reset_index(drop=True)

Pkt reprocessing

In [ ]:
def pkt_reprocessing(df):
  df = df.drop(['time', 'stream_id', 'protocol'], axis=1).reset_index(drop=True)

  # sort by stream_id, protocol, time
  df = df.sort_values(by=['IP_port']).reset_index(drop=True)

  # return 3 series of packet: outgoing, incoming, both
  out_df = df[df['outgoing'] == 1].drop(['outgoing'], axis=1).reset_index(drop=True)
  in_df = df[df['outgoing'] == 0].drop(['outgoing'], axis=1).reset_index(drop=True)
  full_df = df.drop(['outgoing'], axis=1)

  return out_df, in_df, full_df

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def extract_pkt_features(df, type="complete"):
  features_df = df.groupby(['IP_port'], as_index = False).\
    agg({'length':['max', 'min', 'mean', 'mad', 'std', 'var', 'skew', pd.DataFrame.kurt, 'count', 
                   percentile(10), percentile(20), percentile(30), percentile(40), percentile(50), percentile(60), percentile(70), percentile(80), percentile(90)],
     })
    
  # rename columns
  features_df.columns = ['IP_port'] + [type + "_" + x for x in ['max', 'min', 'mean', 'mad', 'std', 'var', 'skew', 'kurt', 'pkt_num', 
                                                                '10per', '20per', '30per', '40per', '50per', '60per', '70per', '80per', '90per']]

    
  return features_df

Flow_based features

In [ ]:
def flow_reprocessing(df):

  df['protocol'] = df['protocol'] == 'tcp'
  df['protocol'] = df['protocol'].astype('int')

  # sort by stream_id, protocol, time
  df = df.sort_values(by=['stream_id', 'protocol', 'time']).reset_index(drop=True)

  # merge packets into flows
  df =  df.groupby(['stream_id', 'protocol', 'IP_port'], as_index = False).\
              agg({'time':['min', 'max'],
                    'length':['sum', 'count']})
  
  df = df.drop(['stream_id'], axis=1)

  df.columns = ['protocol', 'IP_port', 'start', 'end', 'flow_length', 'pkt_num']
  
  # create duration of each flow
  df['duration'] = df['end'] - df['start']
  df = df.drop(['end', 'start'], axis=1)

  return df


def extract_flow_features(df):
  features_df = df.groupby(['IP_port'], as_index = False).\
    agg({'protocol':['mean', 'count'],
         'flow_length': ['mean'],
          'pkt_num': ['mean'],
         'duration': ['mean']
     })
    
  # rename columns
  features_df.columns = ['IP_port', 'protocol', 'flows_num', 'flow_length_mean', 'flow_pkt_num_mean', 'flow_duration_mean']

  return features_df

Weights

In [ ]:
def weights_reprocessing(df):
  df = df.drop(['stream_id', 'protocol', 'length', 'outgoing'], axis=1).reset_index(drop=True)
  return df

Merge pkt and flow features

In [ ]:
def merge_features(df1, df2):
  features_df = pd.merge(df1, df2, on="IP_port")

  # sort by complete pkt number
  features_df = features_df.sort_values(by="complete_pkt_num", ascending=False).reset_index(drop=True)

  return features_df

In [ ]:
def weight(window_indx1, window_indx2):
  intersection = window_indx1.intersection(window_indx2)
  union = window_indx1.union(window_indx2)
  # return len(intersection)/len(union)
  return len(intersection)

def generate_features_weights(df, N, window):
  df = basic_reprocessing(df, N)
  out_df, in_df, complete_df = pkt_reprocessing(df)

  complete_df = extract_pkt_features(complete_df)
  out_df = extract_pkt_features(out_df, "out")
  in_df = extract_pkt_features(in_df, "in")
  pkt_features_df = pd.merge(pd.merge(complete_df, out_df, on="IP_port"), in_df, on="IP_port")
  
  # replace NaN by 0
  pkt_features_df = pkt_features_df.fillna(0)
  
  ################################################
  flow_df = flow_reprocessing(df)
  flow_features_df = extract_flow_features(flow_df)

  ################################################
  features_df = merge_features(pkt_features_df, flow_features_df)
  # features_df = features_df.set_index('IP_port')

  ################################### weights #################################
  w_df = weights_reprocessing(df)
  w_df['time'] = (w_df['time']//window).astype('int')
  w_df = w_df.groupby('IP_port')['time'].agg(active= lambda x: set(x)).reset_index(drop=False)
  
  # create a dataframe of weights
  destination1_list = []
  destination2_list = []
  weight_list = []
  destinations = list(features_df['IP_port'])
  active_destinations = set()

  for i in range(len(destinations)):
    for j in range(i+1, len(destinations)):
      des1 = destinations[i]
      des2 = destinations[j]
      destination1_list.append(des1)
      destination2_list.append(des2)
      w = weight(w_df[w_df['IP_port'] == des1]['active'].values[0], w_df[w_df['IP_port'] == des2]['active'].values[0])
      weight_list.append(w)
      if w > 0:
        active_destinations = active_destinations.union({des1, des2})
  
  # get inactive destinations to remove
  inactive_destinations = list(set(destinations) - active_destinations)
  
  # create dataframe of edge weights
  weights_df = pd.DataFrame(
  {
  "source": destination1_list,
  "target": destination2_list,
  "weight": weight_list,
  }
  )

  weights_df = weights_df.sort_values(by="weight", ascending=False, ignore_index=True)

  # remove destinations that do not connect to any other destination
  features_df = features_df[~features_df['IP_port'].isin(inactive_destinations)]
  # add ip features
  features_df['ip1'] = features_df['IP_port'].apply(lambda x: int(x[0].split('.')[0]))
  features_df['ip2'] = features_df['IP_port'].apply(lambda x: int(x[0].split('.')[1]))
  features_df['ip3'] = features_df['IP_port'].apply(lambda x: int(x[0].split('.')[2]))
  features_df['ip4'] = features_df['IP_port'].apply(lambda x: int(x[0].split('.')[3]))

  weights_df = weights_df[~weights_df['source'].isin(inactive_destinations) & ~weights_df['target'].isin(inactive_destinations)].reset_index(drop=True)

  # min-max normalize weights
  weights_df['weight'] = (weights_df['weight'] - weights_df['weight'].min())/(weights_df['weight'].max() - weights_df['weight'].min())

  return features_df, weights_df

## Generate data

In [ ]:
def generate_graphs_one_app(app_src, saved_graph_folder):
  empty_count = 0

  graphs = []
  count_graph = 0

  ########## create folder to save graphs #############
  for N in N_list:
    for window in window_list:
      saved_graph_Nw = os.path.join(saved_graph_folder, 'N{}/t{}'.format(N, window))
      saved_graph_app = os.path.join(saved_graph_Nw, app_src.split('/')[-1])
      print(saved_graph_app)
      if not os.path.exists(saved_graph_app):
        os.mkdir(saved_graph_app)
  #####################################################

  ########## create a dictionary containing df of graphs ############
  columns1 = ['IP_port', 'complete_max', 'complete_min', 'complete_mean',
       'complete_mad', 'complete_std', 'complete_var', 'complete_skew',
       'complete_kurt', 'complete_pkt_num', 'complete_10per', 'complete_20per',
       'complete_30per', 'complete_40per', 'complete_50per', 'complete_60per',
       'complete_70per', 'complete_80per', 'complete_90per', 'out_max',
       'out_min', 'out_mean', 'out_mad', 'out_std', 'out_var', 'out_skew',
       'out_kurt', 'out_pkt_num', 'out_10per', 'out_20per', 'out_30per',
       'out_40per', 'out_50per', 'out_60per', 'out_70per', 'out_80per',
       'out_90per', 'in_max', 'in_min', 'in_mean', 'in_mad', 'in_std',
       'in_var', 'in_skew', 'in_kurt', 'in_pkt_num', 'in_10per', 'in_20per',
       'in_30per', 'in_40per', 'in_50per', 'in_60per', 'in_70per', 'in_80per',
       'in_90per', 'protocol', 'flows_num', 'flow_length_mean',
       'flow_pkt_num_mean', 'flow_duration_mean', 
       'ip1', 'ip2', 'ip3', 'ip4', 
       'graph_id']
  columns2 = ['source', 'target', 'weight', 'graph_id']
  features_dic = dict()
  weights_dic = dict()
  count_dic = dict()
  for N in N_list:
    for window in window_list:
      features_dic[(N, window)] = pd.DataFrame([], columns=columns1)
      weights_dic[(N, window)] = pd.DataFrame([], columns=columns2)
      count_dic[(N, window)] = 0
  ###################################################################
  
  for filename in os.listdir(app_src):
    path = os.path.join(app_src, filename)
    df = pd.read_csv(path, index_col=0)
    df = df.sort_values(by='time')
      
    if df.empty:
      print("[Empty csv file]")
      empty_count += 1
      continue
        
    df['time'] = df['time'] - df['time'].iloc[0] # get base time

    ######################### generate one graph for all hyper-parameters ###############################
    for N in N_list:
      for window in window_list:
        try:
          node_data, weights = generate_features_weights(df, N, window)
        except:
          print("[Something wrong happen in the process of generating graph from this target dataframe]")
          continue
      
        if weights.shape[0] > 1:
            count_graph += 1
            count_dic[(N, window)] = count_dic[(N, window)] + 1 

            ############################# save graphs ################################
            node_data['graph_id'] = count_dic[(N, window)]
            weights['graph_id'] = count_dic[(N, window)]
            features_dic[(N, window)] = pd.concat([features_dic[(N, window)], node_data], ignore_index=True)
            weights_dic[(N, window)] = pd.concat([weights_dic[(N, window)], weights], ignore_index=True)
            
            ##########################################################################

            if (count_graph % 100 == 0) and count_graph >= 100:
              print('graph :', count_graph)
  

  ########## save 2 .csv files (features and weights) ###########
  for N in N_list:
    for window in window_list:
      
      features_df = features_dic[(N, window)]
      weights_df = weights_dic[(N, window)]

      saved_graph_Nw = os.path.join(saved_graph_folder, 'N{}/t{}'.format(N, window))
      saved_graph_app = os.path.join(saved_graph_Nw, app_src.split('/')[-1])

      features_path = os.path.join(saved_graph_app, 'features.csv')
      weights_path = os.path.join(saved_graph_app, 'weights.csv')

      features_df.to_csv(features_path)
      weights_df.to_csv(weights_path)
      

  ###############################################################
  
  print("================================================================END ONE APP================================================================")

In [ ]:
def generate_graphs(src_folder, saved_graph_folder):
  # build a list of graphs and labels: note that only apply for more than 2 classes
  li = []
  labels = []
  idx = 0

  for app in apps_1:
    idx += 1
    print('Loading {} ... {}/{}'.format(app, idx, len(apps_1 + apps_2 + apps_3 + apps_4 + apps_5 + apps_6 + apps_7 + apps_8)))
    
    app_src = os.path.join(src_folder, app)
    generate_graphs_one_app(app_src, saved_graph_folder)

Generate train and test, combine them together 

In [ ]:
generate_graphs('/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train/',
                '/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train_graphs')

Loading diijam ... 1/101
/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train_graphs/N30/t10/diijam
[Empty csv file]
[Empty csv file]
[Empty csv file]
[Empty csv file]
[Empty csv file]
graph : 100
[Empty csv file]
[Empty csv file]
[Empty csv file]
graph : 200
[Empty csv file]
[Empty csv file]
[Empty csv file]
[Empty csv file]
graph : 300
================================================================END ONE APP================================================================
Loading baomoi ... 2/101
/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train_graphs/N30/t10/baomoi
graph : 100
graph : 200
graph : 300
graph : 400
graph : 500
graph : 600
================================================================END ONE APP================================================================
Loading fptplay ... 3/101
/content/drive/My Drive/AI PROJECTS/GNN/dataset/5_3/train_graphs/N30/t10/fptplay
[Empty csv file]
graph : 100
graph : 200
graph : 300
[Empty csv file]
graph : 400
[Empty cs

___________________________________________________________________________________